In [1]:
import torch as th
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 3
from nnterp import load_model, collect_activations, patchscope_lens, TargetPrompt, collect_activations_batched

In [2]:
base_model = load_model("google/gemma-2-2b")
it_model = load_model("google/gemma-2-2b-it")

In [3]:
def to_question(concept, it_tokenizer=None):
    q = f"What is the meaning of {concept}"
    if it_tokenizer is not None:
        q = it_tokenizer.apply_chat_template(
            [{"role": "user", "content": q}], continue_final_message=True
        )
    return q

In [5]:
from nnterp.utils import UnifiedTransformer
UnifiedTransformer

In [4]:
car_question = to_question("car")
dog_question = to_question("dog")
it_dog_question = to_question("dog", it_tokenizer=it_model.tokenizer)
it_car_question = to_question("car", it_tokenizer=it_model.tokenizer)

base_acts = collect_activations(base_model, [car_question, dog_question])
it_acts = collect_activations(it_model, [it_car_question, it_dog_question])
it_acts_no_template = collect_activations(it_model, [car_question, dog_question])

car_to_dog_base = base_acts[:, 0] - base_acts[:, 1]  # shape: (n_layers, h_dim)
car_to_dog_it = it_acts[:, 0] - it_acts[:, 1]
car_to_dog_it_no_template = it_acts_no_template[:, 0] - it_acts_no_template[:, 1]

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

### Check cosim among the 3 diffs

In [ ]:
# plot mean cosim between the 3 diffs for each layer
cosim_base_it = th.nn.functional.cosine_similarity(car_to_dog_base, car_to_dog_it, dim=1)
cosim_template = th.nn.functional.cosine_similarity(car_to_dog_it, car_to_dog_it_no_template, dim=1)
cosim_base_it_no_template = th.nn.functional.cosine_similarity(car_to_dog_base, car_to_dog_it_no_template, dim=1)
print(cosim_base_it.shape)
print(cosim_template.shape)
print(cosim_base_it_no_template.shape)
fig, ax = plt.subplots()
ax.plot(cosim_base_it.mean(dim=1), label="cosim base it")
ax.plot(cosim_template.mean(dim=1), label="cosim template")
ax.plot(cosim_base_it_no_template.mean(dim=1), label="cosim base it no template")
ax.legend()
plt.show()
